In [21]:
from bs4 import BeautifulSoup     
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import pandas as pd
import math

In [22]:
query_txt=input('크롤링할 키워드는 무엇입니까?: ')
includeQuery=input('결과에서 반드시 포함하는 단어를 입력하세요: ')
decludeQuery=input('결과에서 제외할 단어를 입력하세요: ')
startTime=input('조회 시작일자 입력(예:2019-01-01): ')
endTime=input('조회 종료일자 입력(예:2019-03-31): ')
count=int(input('크롤링할 건수는 몇건입니까?: '))
f_name = input('파일로 저장할 폴더 이름을 쓰세요(예:c:\data\): ')


pagecount=math.ceil(int(count)/7)

num=1
numList=[]
blogLinks=[]
nickNames=[]
wDates=[]
contents=[]

In [23]:
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.documentElement.scrollHeight);")
    time.sleep(0.5)

In [24]:
path = '/opt/homebrew/Caskroom/chromedriver/112.0.5615.49/chromedriver'
driver = webdriver.Chrome(path)
driver.get("https://section.blog.naver.com/BlogHome.naver?directoryNo=0&currentPage=1&groupId=0")
time.sleep(2)

C:\Users\lego8\AppData\Local\Temp\ipykernel_6400\44218747.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [25]:
element=driver.find_element(By.NAME, 'sectionBlogQuery').click()
driver.find_element(By.XPATH, '//*[@id="header"]/div[1]/div/div[2]/form/fieldset/div/input').send_keys(query_txt+' +'+includeQuery+' -'+decludeQuery)

element=driver.find_element(By.XPATH, '//*[@id="header"]/div[1]/div/div[2]/form/fieldset/a[1]').click()
time.sleep(1)

In [26]:
element=driver.find_element(By.CLASS_NAME, 'present_selected').click()
time.sleep(0.5)
driver.find_element(By.XPATH, '//*[@id="search_start_date"]').send_keys(startTime)
driver.find_element(By.XPATH, '//*[@id="search_end_date"]').send_keys(endTime)
element=driver.find_element(By.ID, 'periodSearch').click()
time.sleep(1)

In [27]:
def addData(content_list):
    global num,numList,blogLinks,nickNames,wDates,contents
    for i in content_list:
        numList.append(num)
        
        blogLink=i.find('div','info_post').find('a','desc_inner').get('href')
        blogLinks.append(blogLink)

        nickName=i.find('div','info_post').find('em','name_author').get_text( )
        nickNames.append(nickName)
        
        writeDate=i.find('div','info_post').find('span','date').get_text( )
        wDates.append(writeDate)

        content=i.find('div','info_post').find('a','text').get_text( )
        contents.append(content)

        num+=1

In [28]:
for i in range(pagecount):
    scroll_down(driver)
    full_html = driver.page_source
    soup = BeautifulSoup(full_html, 'html.parser')

    content_list = soup.findAll('div',class_='list_search_post')
    addData(content_list)

    element=driver.find_element(By.XPATH, '//*[@id="content"]/section/div[3]/span[{}]/a'.format((i%10)+2)).click()
    time.sleep(1)

driver.quit()

In [29]:
dataFrame=pd.DataFrame()
dataFrame['']=numList[:count]
dataFrame['블로그 주소']=blogLinks[:count]
dataFrame['댓글작성자명']=nickNames[:count]
dataFrame['댓글작성일자']=wDates[:count]
dataFrame['댓글 내용']=contents[:count]

dataFrame.to_csv(f_name+'Q2.csv',encoding="utf-8-sig",index=False)
dataFrame.to_excel(f_name+'Q2.xlsx',index=False)

f=open(f_name+'Q2.txt','a',encoding='UTF-8')
f.write(str(numList))
f.write(str(blogLinks))
f.write(str(nickNames))
f.write(str(wDates))
f.write(str(contents))
f.close( )